In [22]:
import pandas as pd 
import numpy as np 
import requests
import matplotlib as plt 
import seaborn as sns 
import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [60]:
url = "http://www.isthe.com/chongo/tech/astro/HR-temp-mass-table-byhrclass.html#below"

In [107]:
star_data = pd.DataFrame()
html = pd.read_html(url)
for i in html:
    star_data = pd.concat([star_data, i])

In [96]:
len(star_data)

1242

In [108]:
star_data.drop([0, 1, 2], axis=1, inplace=True)

In [109]:
star_data.dropna(axis=0, inplace=True)

In [110]:
len(star_data)

1100

In [111]:
for index, row in star_data.iterrows():
    temp = row['Star ColorRGB 0-255'].split()
    int_list = [int(i) for i in temp]
    int_tup = tuple(int_list)
    hex_col = '%02x%02x%02x' % int_tup
    star_data['hex_color'] = int(hex_col, 16)

In [112]:
star_data.drop(['Star ColorRGB 0-255'], axis=1, inplace=True)

In [116]:
star_data.head(50)

,Abs MagMv,Bolo CorrBC(Temp),Bolo MagMbol,Color IndexB-V,LuminosityLstar/Lsun,MassMstar/Msun,RadiusRstar/Rsun,StellarType,TempK,hex_color
1182,10.2,-7.55,2.65,-0.37,6.910000,1.1,0.00902,DB0,100000.0,16773089
1183,10.8,-4.61,6.19,-0.35,0.265000,0.9,0.00696,DB1,50400.0,16773089
1184,11.4,-2.67,8.73,-0.27,0.025500,0.8,0.00864,DB2,25200.0,16773089
1185,11.9,-1.60,10.30,-0.20,0.006020,0.7,0.00944,DB3,16800.0,16773089
1186,12.5,-0.90,11.60,-0.13,0.001820,0.6,0.00922,DB4,12600.0,16773089
1187,13.1,-0.45,12.65,-0.07,0.000693,0.5,0.00890,DB5,10080.0,16773089
1188,13.7,-0.20,13.50,0.09,0.000315,0.4,0.00864,DB6,8400.0,16773089
1189,14.2,-0.09,14.11,0.34,0.000180,0.3,0.00889,DB7,7200.0,16773089
1190,14.8,-0.10,14.70,0.55,0.000105,0.2,0.00887,DB8,6300.0,16773089
1191,15.4,-0.22,15.18,0.74,0.000067,0.1,0.00898,DB9,5600.0,16773089


In [106]:
star_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 1 to 1235
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Abs MagMv             1100 non-null   float64
 1   Bolo CorrBC(Temp)     1100 non-null   float64
 2   Bolo MagMbol          1100 non-null   float64
 3   Color IndexB-V        1100 non-null   float64
 4   LuminosityLstar/Lsun  1100 non-null   float64
 5   MassMstar/Msun        1100 non-null   float64
 6   RadiusRstar/Rsun      1100 non-null   float64
 7   StellarType           1100 non-null   object 
 8   TempK                 1100 non-null   float64
 9   hex_color             1100 non-null   object 
dtypes: float64(8), object(2)
memory usage: 134.5+ KB


In [10]:
import pandas_profiling

In [25]:
profile = star_data.profile_report(title='EDA of star dataset using Pandas Profiling')
# save the minified report
profile.to_file(output_file="star.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 59.52it/s]


In [12]:
# Set up x and y variables
X = star_data[['Abs MagMv', 'Bolo CorrBC(Temp)', 'Bolo MagMbol', 'Color IndexB-V', 'LuminosityLstar/Lsun', 'MassMstar/Msun', 'RadiusRstar/Rsun', 'Star ColorRGB 0-255', 'TempK']]
y = star_data['StellarType']

In [16]:
# Make training and testing set (testing is 20% of the data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=14)

In [40]:
lr = LogisticRegression()
lr.fit(X,y)

ValueError: could not convert string to float: '255 239 225'

TODO:   
-Logistic regression   
-Support vector machine  
-Naive Bayes

In [26]:
test = "225 332 999"
print(test)

225 332 999


In [34]:
test = test.split()

In [35]:
test_list = [int(i) for i in test] 

In [36]:
test_list

[225, 332, 999]

In [47]:
star_data['rgb_list'] = [[x.split() for x in star_data['Star ColorRGB 0-255']int(z) for z in star_data['Star ColorRGB 0-255']]]

SyntaxError: invalid syntax (<ipython-input-47-878a35994fab>, line 1)

In [42]:
def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb

In [44]:
star_data['hex_color'] = [rgb_to_hex(x) for x in star_data['rgb_list']]

TypeError: %x format: an integer is required, not list

In [45]:
tester = (256,567,380

In [46]:
type(tester)

tuple

In [50]:
star_data.head()

,Abs MagMv,Bolo CorrBC(Temp),Bolo MagMbol,Color IndexB-V,LuminosityLstar/Lsun,MassMstar/Msun,RadiusRstar/Rsun,Star ColorRGB 0-255,StellarType,TempK,rgb_list,hex_color
1,-9.5,-4.58,-14.08,-0.35,34100000.0,160.0,80.2,144 166 255,O0Ia0,50000.0,"[144, 166, 255]",ffefe1
2,-6.7,-4.58,-11.28,-0.35,2590000.0,150.0,22.1,144 166 255,O0Ia,50000.0,"[144, 166, 255]",ffefe1
3,-6.5,-4.58,-11.08,-0.35,2150000.0,140.0,20.2,144 166 255,O0Ib,50000.0,"[144, 166, 255]",ffefe1
4,-6.5,-4.58,-11.08,-0.35,2150000.0,130.0,20.2,144 166 255,O0II,50000.0,"[144, 166, 255]",ffefe1
5,-6.5,-4.58,-11.08,-0.35,2150000.0,120.0,20.2,144 166 255,O0III,50000.0,"[144, 166, 255]",ffefe1
